# Implementação Chatbot para respostas automatizadas com Bert Base

Importação de bibliotecas

In [ ]:
import time
# Registrar o tempo de início
tempo_inicio = time.time()

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

import pandas as pd
import json

import requests
from io import StringIO

print(torch.__version__)

import sys
from transformers import __version__ as transformers_version
import transformers as transformers

print("Versão do Python", sys.version)
print("Versão do json:", json.__version__)
print("Versão do requests:", requests.__version__)
print("Versão do numpy:", np.__version__)
print("Versão do pandas:", pd.__version__)
print("Versão do torch:", torch.__version__)
print("Versão do transformers:", transformers.__version__)


2.1.0+cu121
Versão do Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Versão do json: 2.0.9
Versão do requests: 2.31.0
Versão do numpy: 1.25.2
Versão do pandas: 1.5.3
Versão do torch: 2.1.0+cu121
Versão do transformers: 4.38.1


#Importação de dados

In [ ]:
#importação dos dados
url = 'https://raw.githubusercontent.com/katiacardoso/EMAP_Chatbot/main/perguntas_respostass.csv'

# Baixar o conteúdo do arquivo usando requests
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    # Criar um objeto pandas DataFrame a partir do conteúdo baixado
    df = pd.read_csv(StringIO(response.text), usecols=['Pergunta', 'Resposta'])
    # Agora você pode usar o DataFrame 'data' normalmente
else:
    print("Falha ao baixar o arquivo. Código de status:", response.status_code)


df.to_csv('perguntas_respostass.csv', index=False)

In [ ]:

# Carregue o modelo BERT pré-treinado e o tokenizador
model_name = "bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)



#Transformação em CSV

In [ ]:

# Carregue o arquivo CSV com perguntas e respostas
#df = pd.read_csv("perguntas_respostass.csv")

# Converter os dados para um formato adequado para JSON
data = []
for index, row in df.iterrows():
    pergunta = row['Pergunta']
    resposta = row['Resposta']
    data.append({'pergunta': pergunta, 'resposta': resposta})


# Salvar os dados como JSON em um arquivo
with open('perguntas_respostas.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

# Carregar dados do arquivo JSON
with open('perguntas_respostas.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Extrair perguntas e respostas
perguntas = [qa['pergunta'] for qa in data]
respostas = [qa['resposta'] for qa in data]

#Vetorização

In [ ]:
# Função para vetorizar texto usando BERT
def vetorizar_texto(texto):
    inputs = tokenizer(texto, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Média das embeddings das palavras
    return embeddings

# Vetorizar perguntas usando BERT
perguntas_embeddings = torch.cat([vetorizar_texto(pergunta) for pergunta in perguntas])

In [ ]:
perguntas_embeddings.shape

torch.Size([120, 768])

#Testes de entendimento de codigo

In [ ]:

'''# Print do número de perguntas vetorizadas
print(len(perguntas_embeddings))

# Imprimir as embeddings das perguntas
#print(perguntas_embeddings)


# Imprimir as embeddings das palavras individuais
for i, pergunta in enumerate(perguntas):
    print("Pergunta:", pergunta)
    print("Embeddings:", perguntas_embeddings[i])

# Selecionar uma pergunta específica
#pergunta_selecionada = perguntas[0]
pergunta_selecionada = 'como chegar ao Porto?'
#pergunta_selecionada = input("Insira uma pergunta: ")

# Vetorizar a pergunta selecionada
pergunta_embeddings = vetorizar_texto(pergunta_selecionada)

# Imprimir a pergunta selecionada
print(pergunta_selecionada)

# Imprimir as embeddings da pergunta selecionada
#print(pergunta_embeddings)
#neste codigo acima imprime um embedding gigante

# Vetorizar as perguntas usando BERT
perguntas_embeddings = torch.cat([vetorizar_texto(pergunta) for pergunta in pergunta_selecionada])

# Imprimir as embeddings das perguntas
print(perguntas_embeddings)'''



'# Print do número de perguntas vetorizadas\nprint(len(perguntas_embeddings))\n\n# Imprimir as embeddings das perguntas\n#print(perguntas_embeddings)\n\n\n# Imprimir as embeddings das palavras individuais\nfor i, pergunta in enumerate(perguntas):\n    print("Pergunta:", pergunta)\n    print("Embeddings:", perguntas_embeddings[i])\n\n# Selecionar uma pergunta específica\n#pergunta_selecionada = perguntas[0]\npergunta_selecionada = \'como chegar ao Porto?\'\n#pergunta_selecionada = input("Insira uma pergunta: ")\n\n# Vetorizar a pergunta selecionada\npergunta_embeddings = vetorizar_texto(pergunta_selecionada)\n\n# Imprimir a pergunta selecionada\nprint(pergunta_selecionada)\n\n# Imprimir as embeddings da pergunta selecionada\n#print(pergunta_embeddings)\n#neste codigo acima imprime um embedding gigante\n\n# Vetorizar as perguntas usando BERT\nperguntas_embeddings = torch.cat([vetorizar_texto(pergunta) for pergunta in pergunta_selecionada])\n\n# Imprimir as embeddings das perguntas\nprint

#tokenização

In [ ]:
def chatbot(user_input, limiar_minimo=0.2):
    user_input_embedding = vetorizar_texto(user_input)
    similarity_scores = cosine_similarity(user_input_embedding, perguntas_embeddings)
    average_similarity = similarity_scores.mean()

    # Adicione um limiar de similaridade mínimo para classificar a resposta
    if average_similarity > limiar_minimo:
        best_match_index = similarity_scores.argmax()
        resposta = respostas[best_match_index]
    else:
        resposta = "Desculpe, não consegui encontrar uma correspondência adequada."

    return resposta

In [ ]:
# Função para encontrar as melhores respostas com base na pergunta do usuário usando BERT
def chatbot_verbose(user_input, limiar_minimo=0.7, top_n=5):
    user_input_embedding = vetorizar_texto(user_input)
    similarity_scores = cosine_similarity(user_input_embedding, perguntas_embeddings)
    top_n_indices = similarity_scores.flatten().argsort()[-top_n:][::-1]

    # Imprime as 5 perguntas mais similares e seus valores de similaridade
    print("Top 5 Perguntas Mais Similares:")
    for i, index in enumerate(top_n_indices):
        pergunta_similar = perguntas[index]
        similaridade = similarity_scores[0, index]
        print(f"{i+1}. {pergunta_similar} - Similaridade: {similaridade}")
    # Adiciona um limiar de similaridade mínimo para classificar as respostas
    if similarity_scores[0, top_n_indices[0]] > limiar_minimo:
        best_match_index = top_n_indices[0]
        pergunta_mais_similar = perguntas[best_match_index]
        resposta_correspondente = respostas[best_match_index]
        print(f"\nPergunta mais semelhante (índice {best_match_index}): {pergunta_mais_similar}")
        print(f"Resposta correspondente: {resposta_correspondente}")
    else:
        resposta_correspondente = "Desculpe, não consegui encontrar uma correspondência adequada."
        print(resposta_correspondente)

    return resposta_correspondente



# Pergunta do usuário
user_question = "Capitu traiu Bentinho?"

# Obtenha a resposta usando chatbot_verbose
response = chatbot_verbose(user_question)

# Imprima a resposta
#print("Resposta do Chatbot:", response)

Top 5 Perguntas Mais Similares:
1. Como é o processo de seleção e entrevista para empregos no Porto do Itaqui? - Similaridade: 0.6952014565467834
2. Como é o processo de seleção e entrevista para empregos no Porto do Itaqui? - Similaridade: 0.6952014565467834
3. É necessário agendar previamente a visita ao Porto do Itaqui? - Similaridade: 0.6790130138397217
4. Quais rodovias principais levam ao Porto do Itaqui? - Similaridade: 0.6641402244567871
5. Como posso solicitar uma entrevista com representantes do Porto do Itaqui para minha reportagem? - Similaridade: 0.6601680517196655
Desculpe, não consegui encontrar uma correspondência adequada.


A pergunta "Capitu traiu Bentinho" teve maior similaridade com perguntas da base do  que "Como chegar ao Porto". Pergunta para uma próxima conversa com o João : Como delimitar que o chat responda apenas perguntas referentes aquele assunto?


In [ ]:
 # Calcular o tempo de execução
tempo_execucao = time.time() - tempo_inicio
print("Tempo de execução do chatbot: {:.2f} segundos".format(tempo_execucao))

Tempo de execução do chatbot: 45.22 segundos


#simulaçao de conversa

In [ ]:
# Lista para armazenar as perguntas e respostas
conversa = []

# Inicie uma conversa com o chatbot
while True:
    user_input = input("Você: ")
    if user_input.lower() == "sair":
        print("Chatbot: Adeus! Até logo.")
        break
    response = chatbot(user_input)
    #response = chatbot(user_input)
    print("Chatbot:", response)

    # Armazene a pergunta do usuário e a resposta do chatbot na lista
    conversa.append({'pergunta': user_input, 'resposta': response})

    # Salve a lista como JSON em um arquivo
    with open('conversa.json', 'a', encoding='utf-8') as json_file:
        json.dump(conversa, json_file, ensure_ascii=False, indent=4)

Você: Como chegar no Porto?
Chatbot: Confira aqui o mapa de localização e seja bem-vindo! http://www.portodoitaqui.ma.gov.br/porto-do-itaqui/como-chegar 


In [ ]:
# Pergunta do usuário
user_question = "Como chegar no Porto"

# Obtenha a resposta usando chatbot_verbose
response = chatbot_verbose(user_question)

# Imprima a resposta
print("Resposta do Chatbot:", response)